In [1]:
import PyPDF2
import glob
import numpy as np
import pdfplumber
from collections import defaultdict
from pdfminer.high_level import extract_pages, extract_text
from pdfminer.layout import LTTextContainer, LTChar, LTRect, LTFigure

In [2]:
def text_extraction(element):
    
    line_text = element.get_text()
    
    line_formats = []
    for text_line in element:
        
        if isinstance(text_line, LTTextContainer):
            
            for character in text_line:
                
                if isinstance(character, LTChar):
                    
                    line_formats.append(character.fontname)
                    
                    line_formats.append(character.size)
                    
    format_per_line = list(set(line_formats))
    
    return (line_text, format_per_line)

def analyst_executives(path):
    
    time=0
    presentation_index=0
    Elon_Musk_index=0
    Question_and_Answer_index=0
    Elon_Musk_speech=[]
    presentation_speech=[]
    text=[]
    executives_name=[]
    analysts_name=[]

    executives_title=[]
    analysts_title=[]

    executives_index=0
    analysts_index=0

    for pagenum, page in enumerate(extract_pages(path)):
        
        for _, element in enumerate(page):

            if pagenum == 2:

                if isinstance(element, LTTextContainer):
                    line_text, format_per_line = text_extraction(element)

                text.append(line_text)

                if line_text == "EXECUTIVES\n":
                    executives_index+=1

                if line_text == 'ANALYSTS\n':
                    executives_index=0

                if executives_index >= 1:
                    executives_name.append(line_text.split("\n")[0])
                    executives_title.append(line_text.split("\n")[1])

                if line_text == 'ANALYSTS\n':
                    analysts_index+=1
                    executives_index=0

                if 'S&P Global Market Intelligence' in line_text:
                    analysts_index=0

                if analysts_index >= 1:
                    analysts_name.append(line_text.split("\n")[0])    
                    analysts_title.append(line_text.split("\n")[1])

        if pagenum == 4:
            break

    ppl=defaultdict(lambda: defaultdict(str))

    for i in range(1, len(analysts_name), 1):

        ppl['analysts'][analysts_name[i]]=analysts_title[i]

    for i in range(1, len(executives_name), 1):

        ppl['executives'][executives_name[i]]=executives_title[i]
    
    return ppl

In [74]:
def processing_qa(path):
    
    presentation_index=0
    Elon_Musk_index=0
    Question_and_Answer_index=0
    Elon_Musk_speech=[]
    question_and_answer_speech=[]
    time=0

    for pagenum, page in enumerate(extract_pages(path)):
        
        print('page={}'.format(pagenum, page))
        
        for _, element in enumerate(page):
            
            if isinstance(element, LTTextContainer):

                line_text, format_per_line = text_extraction(element)
            
            if pagenum == 0:
                
                if _ == 0:
                    
                    time=line_text
                    time=time[61:100]
            
            if "TESLA, INC."  in line_text:
        
                pass
        
            elif "S&P Global Market Intelligence" in line_text:
        
                pass
            
            elif len(line_text) < 6:
        
                pass
        
            else:

                if line_text == 'Presentation\n':

                    presentation_index = presentation_index + 1

                if line_text == 'Question and Answer\n':

                    Question_and_Answer_index = Question_and_Answer_index + 1

                if Question_and_Answer_index == 2:

                    question_and_answer_speech.append(line_text)

                if pagenum == pdfReader.pages.length_function()-1:

                    break
                
    return question_and_answer_speech

In [225]:
def questions_answers_dict(question_and_answer_speech, executives_people, analysts_people):

    speech_analysts_people=defaultdict(lambda: defaultdict(str))
    speech_executives_people=defaultdict(lambda: defaultdict(str))
    speech_analysts_index_dict=defaultdict(lambda: defaultdict(str))
    speech_executives_index_dict=defaultdict(lambda: defaultdict(str))
    speech_question_index=0
    speech_answer_index=0
        
    for _, text in enumerate(question_and_answer_speech):

        text=text.split("\n")

        if "S&P Global Market Intelligence" in text[0]:

            pass

        else:

            for analysts in analysts_people:
                                
                if analysts in text[0]:
                                        
                    speech_analysts_people[str("question_")+str(speech_question_index)]=analysts
                    speech_analysts_index_dict[str("question_")+str(speech_question_index)]=_+1
                    speech_question_index+=1
                        
            for executives in executives_people:

                if executives in text[0]:

                    speech_executives_people[str("answer_")+str(speech_answer_index)]=executives
                    speech_executives_index_dict[str("answer_")+str(speech_answer_index)]=_+1
                    speech_answer_index+=1
    
    # ================================================================================ 
    
    speech_speech_executives_index_list=[]

    for i in speech_executives_index_dict.items():

        speech_speech_executives_index_list.append(i[1])

    speech_speech_executives_index_list=np.array(speech_speech_executives_index_list)+1

    # ================================================================================
    
    speech_speech_analysts_index_list=[]

    for i in speech_analysts_index_dict.items():
            
        speech_speech_analysts_index_list.append(i[1])

    speech_speech_analysts_index_list=np.array(speech_speech_analysts_index_list)+1
        
    # ================================================================================
    
    speech_speech_executives_dict=defaultdict(lambda: defaultdict(list))

    for i in speech_executives_index_dict.items():
        
        if i[1] == len(question_and_answer_speech):
            break
            
        speech_speech_executives_dict[i[0]][i[1]].append(question_and_answer_speech[i[1]])
    
    # ================================================================================ 
    
    speech_speech_analysts_dict=defaultdict(lambda: defaultdict(list))

    for i in speech_analysts_index_dict.items():
        
        if i[1] == len(question_and_answer_speech):
            break
            
        speech_speech_analysts_dict[i[0]][i[1]].append(question_and_answer_speech[i[1]])
    
    
    return speech_speech_executives_dict,\
            speech_speech_analysts_dict,\
            speech_executives_people,\
            speech_analysts_people



In [207]:
files=glob.glob("/Volumes/4TB/ConferenceCall/Tesla/Earningcall/*.pdf")
path=files[-3]
print(path)
pdf=open(path, 'rb')
pdfReader=PyPDF2.PdfReader(pdf)

question_and_answer_speech=processing_qa(path)

ppl=analyst_executives(path)

ppl_reverse={v:k  for k,v in ppl['executives'].items()}

for i in ppl_reverse:

    if "Investor" in i:
                
        ppl['analysts'][ppl_reverse[i]]=i
        
        ppl['executives'].pop(ppl_reverse[i], None)

executives_people=[elem[0] for elem in ppl['executives'].items()]
executives_title=[elem[1] for elem in ppl['executives'].items()]

analysts_people=[elem[0] for elem in ppl['analysts'].items()]
analysts_title=[elem[1] for elem in ppl['analysts'].items()]

/Volumes/4TB/ConferenceCall/Tesla/Earningcall/Tesla, Inc., Q4 2021 Earnings Call, Jan 26, 2022.pdf
page=0
page=1
page=2
page=3
page=4
page=5
page=6
page=7
page=8
page=9
page=10
page=11
page=12
page=13
page=14
page=15
page=16
page=17
page=18
page=19


In [208]:
ppl

defaultdict(<function __main__.analyst_executives.<locals>.<lambda>()>,
            {'analysts': defaultdict(str,
                         {'A.M. Sacconaghi': 'Sanford C. Bernstein & Co., LLC.,',
                          'Benjamin Joseph Kallo': 'Robert W. Baird & Co.',
                          'Jonathan Edward Dorsheimer': 'Canaccord Genuity Corp.,',
                          'Pierre C. Ferragu': 'New Street Research LLP',
                          'Martin Viecha': 'Senior Director for Investor'}),
             'executives': defaultdict(str,
                         {'Andrew D. Baglino': 'Senior Vice President of Powertrain',
                          'Elon R. Musk': 'Technoking of Tesla, CEO &',
                          'Lars Moravy': 'Vice President of Vehicle',
                          'Zachary J. Kirkhorn': 'Master of Coin & CFO'})})

In [232]:
speech_speech_executives_dict,\
speech_speech_analysts_dict,\
speech_executives_people,\
speech_analysts_people=\
questions_answers_dict(question_and_answer_speech,
                       executives_people,
                       analysts_people)

In [235]:
speech_speech_executives_dict

defaultdict(<function __main__.questions_answers_dict.<locals>.<lambda>()>,
            {'answer_0': defaultdict(list,
                         {4: ["Well, we're not currently working on the $25,000 car. At some point, we will. But we have enough on our\nplate right now, too much on our plate, frankly. So at some point there, we'll be. I think that's sort of a\nquestion that is sort of the wrong question really. Really, the thing that overwhelmingly matters is when is\nthe car autonomous. At the point in which it is autonomous, the cost of transport drops by, I don't know, a\nfactor of 4 or 5.\n"]}),
             'answer_1': defaultdict(list,
                         {8: ['Do you want to talk about that, Drew?\n']}),
             'answer_2': defaultdict(list,
                         {10: ["Yes. I think from a mission perspective, it's very aligned. If you imagine replacing natural gas, water and\nspace heaters with electric heat pumps, it offsets something equivalent to like 80% of wh

In [236]:
speech_speech_analysts_dict

defaultdict(<function __main__.questions_answers_dict.<locals>.<lambda>()>,
            {'question_0': defaultdict(list,
                         {2: ["Thank you very much. Let's go to the Q&A from the investor side. The first question was on 4680 cells,\nwhich we already answered, so let's go to the second question. How is the progress of the $25,000\ncompact car? Can you give an update?\n"]}),
             'question_1': defaultdict(list,
                         {6: ["Thank you. The next question from investors is, since we're talking product road maps today, how do you\nview domestic cooling and heating in the context of accelerating the sustainable energy transition? And\nhow might Tesla's HVAC and heat pump advances fit it?\n"]}),
             'question_2': defaultdict(list,
                         {40: ['The next question is, would you consider splitting FSD packages into perpetual and term licenses with\na higher tier for both options for commercial use? A perpetual license cou